In [1]:
import numpy as np
import pandas as pd

import json

from datetime import datetime, timedelta

In [2]:
features = pd.read_excel('features.xlsx')

In [3]:
# print(features)

## Task Description

In [4]:
for i in range(len(features)):
    print(features['Logic'][i], '\n', features['If missing value'][i])
    print()
    print('************')

Description: number of claims for last 180 days
Source: contracts
Key fields: claim_id, claim_date
Special notes: 
1. In case claim date is null, don't take into consideration such claims. 
 In case no claims, then put -3 as a value of this feature.

************
Description: Sum of exposue of loans without TBC loans. Exposure means here field "loan_summa".
Source: contracts
Key fields: bank, loan_summa, contract_date
Special notes: 
1. Consider only loans where field "bank" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].
2. Disbursed loans means loans where contract_date is not null 
 In case no claims, then put -3 as a value of this feature.
If no loans at all, then put -1 as a value of this feature.

************
Description: Number of days since last loan. 
Source: contracts
Key fields: contract_date, summa
Special notes:
1. Take last loan of client where summa is not null and calculate number of days from contract_date of this loan to application date. 
 In case no claims at all, th

## Useful functions

In [5]:
# replace '' strings with np.nan if any exists
# drop np.nan values and reindex the dataframe
def clean_data(df, column_name):
    df.replace('', np.nan, inplace=True)
    df.dropna(subset=column_name, inplace=True)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [6]:
def flatten_contracts(df, column_name, contracts_list = []):
    for contracts in df[column_name]:
        if isinstance(contracts, list):
            for contract in contracts:
                contracts_list.append(contract)
        elif isinstance(contracts, dict):
            contracts_list.append(contracts)
    
    return contracts_list

In [7]:
# convert a list of dictionaries to dataframe
def list_to_dataframe(list_of_data):
    df = pd.DataFrame(list_of_data)
    return df

In [8]:
def convert_to_datetime(df, column_names):
    if isinstance(column_names, list):
        for column_name in column_names:
            df[column_name] = pd.to_datetime(df[column_name])
    else:
        df[column_names] = pd.to_datetime(df[column_names])
    return df

## Loading and cleaning the data

In [9]:
data = pd.read_csv("data.csv")
data.head()

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN


In [10]:
# Remove Nan values
clean_data(data, ['contracts']).head()

,id,application_date,contracts
0,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
1,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925215.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
3,2925216.0,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925218.0,2024-02-12 19:40:15.507000+00:00,"[{""contract_id"": """", ""bank"": ""062"", ""summa"": ""..."


In [11]:
# json loads
data['contracts'] = data['contracts'].apply(lambda x: json.loads(x) if pd.notnull(x) else np.nan)
data.head()

,id,application_date,contracts
0,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa..."
1,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa..."
2,2925215.0,2024-02-12 19:25:10.176000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa..."
3,2925216.0,2024-02-12 19:25:10.176000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa..."
4,2925218.0,2024-02-12 19:40:15.507000+00:00,"[{'contract_id': '', 'bank': '062', 'summa': '..."


In [12]:
# flatten_contracts(data, 'contracts')

In [13]:
# create dataframe using 'contracts' value
contracts_df = list_to_dataframe(flatten_contracts(data, 'contracts'))
contracts_df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530,003,500000000,0,13.02.2020,609965,17.02.2020
1,,014,,,28.08.2020,F00013731,
2,,014,,,08.10.2020,F00021301,
3,,014,,,25.11.2020,F00037907,
4,,053,,,09.12.2020,34852,


In [14]:
contracts_df.shape

(9270, 7)

In [15]:
# # clean the dataframe and remove empty values
# df_cleaned = clean_data(contracts_df, column_name=['contract_id'])
# df_cleaned.head()  

In [16]:
# df_cleaned.shape

In [17]:
print(type(contracts_df['claim_date'][0]))
print(type(contracts_df['contract_date'][0]))

<class 'str'>
<class 'str'>


In [18]:
columns_to_clean = ['claim_date', 'contract_date']
convert_to_datetime(contracts_df, columns_to_clean)
contracts_df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530,003,500000000,0,2020-02-13,609965,2020-02-17
1,,014,,,2020-08-28,F00013731,NaT
2,,014,,,2020-08-10,F00021301,NaT
3,,014,,,2020-11-25,F00037907,NaT
4,,053,,,2020-09-12,34852,NaT


In [19]:
print(type(contracts_df['claim_date'][0]))
print(type(contracts_df['contract_date'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


### number of claims for last 180 days

In [20]:
print(features['Logic'][0], '\n', features['If missing value'][0])

Description: number of claims for last 180 days
Source: contracts
Key fields: claim_id, claim_date
Special notes: 
1. In case claim date is null, don't take into consideration such claims. 
 In case no claims, then put -3 as a value of this feature.


In [21]:
df_claims = contracts_df.drop_duplicates(subset='claim_id')

In [22]:
# difference between the claim_date and today's date in days
df_claims['days_since_claim'] = (datetime.today() - df_claims['claim_date']).dt.days

<ipython-input-22-5dcd51b7a0f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['days_since_claim'] = (datetime.today() - df_claims['claim_date']).dt.days


In [23]:
df_claims['claim_flag'] = [1 if days <= 180 else 0 for days in df_claims['days_since_claim']]

<ipython-input-23-94f51db42769>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['claim_flag'] = [1 if days <= 180 else 0 for days in df_claims['days_since_claim']]


In [24]:
print(features['Feature'][0])

tot_claim_cnt_l180d


In [25]:
tot_claim_cnt_l180d = [df_claims['claim_flag'].sum() if df_claims['claim_flag'].sum() > 0 else -3]
print('number of claims for last 180 days:', tot_claim_cnt_l180d[0])

number of claims for last 180 days: 117


### Sum of exposue of loans

In [26]:
print(features['Logic'][1], '\n', features['If missing value'][1])

Description: Sum of exposue of loans without TBC loans. Exposure means here field "loan_summa".
Source: contracts
Key fields: bank, loan_summa, contract_date
Special notes: 
1. Consider only loans where field "bank" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].
2. Disbursed loans means loans where contract_date is not null 
 In case no claims, then put -3 as a value of this feature.
If no loans at all, then put -1 as a value of this feature.


In [27]:
# field "bank" is not in ['LIZ', 'LOM', 'MKO', 'SUG']
df_loans = contracts_df[~contracts_df['bank'].isin(['LIZ', 'LOM', 'MKO', 'SUG'])]
# null values should be handled separately
df_loans = df_loans.dropna(subset=['bank'])

In [28]:
# remove rows from df_loans dataframe where contract_date is null
clean_data(df_loans, column_name=['contract_date'])

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530.0,003,5.000000e+08,0.000000e+00,2020-02-13,609965,2020-02-17
1,35163.0,053,5.100000e+08,0.000000e+00,2020-12-15,35163,2020-12-21
2,500873.0,055,5.000000e+08,0.000000e+00,2022-09-12,451262,2022-12-12
3,95511.0,004,2.800000e+09,1.826667e+09,2022-08-11,95511,2022-08-11
4,414955.0,011,5.000000e+08,9.502356e+07,2023-05-04,2411695,2023-05-04
...,...,...,...,...,...,...,...
351,12144992.0,005,1.930000e+09,1.177375e+09,2022-10-21,3382076,2022-10-21
352,553516633.0,063,4.600000e+08,4.227725e+08,2023-09-10,553516633,2023-09-10
353,79365.0,014,3.400000e+08,3.305556e+08,2023-12-26,79365,2023-12-26
354,18410.0,004,4.916470e+07,0.000000e+00,2019-03-04,18410,2019-03-04


In [29]:
print(features['Feature'][1])

disb_bank_loan_wo_tbc


In [30]:
df_loans['loan_summa'].dtype

dtype('float64')

In [31]:
disb_bank_loan_wo_tbc = df_loans['loan_summa'].sum()

if disb_bank_loan_wo_tbc == 0:
    disb_bank_loan_wo_tbc = -1
elif disb_bank_loan_wo_tbc == np.nan:
    disb_bank_loan_wo_tbc = -3

print('Sum of exposue of loans without TBC loans:', disb_bank_loan_wo_tbc)

Sum of exposue of loans without TBC loans: 303717438573.0


### Number of days since last loan

In [32]:
print(features['Logic'][2], '\n', features['If missing value'][2])

Description: Number of days since last loan. 
Source: contracts
Key fields: contract_date, summa
Special notes:
1. Take last loan of client where summa is not null and calculate number of days from contract_date of this loan to application date. 
 In case no claims at all, then put -3 as a value of this feature.
In case no loans at all, then put -1 as a value of this feature.


In [33]:
# remove rows where 'summa' is nan
loans_df = clean_data(contracts_df, ['summa'])
loans_df.head()

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,522530,003,5.000000e+08,0.000000e+00,2020-02-13,609965,2020-02-17
1,35163,053,5.100000e+08,0.000000e+00,2020-12-15,35163,2020-12-21
2,500873,055,5.000000e+08,0.000000e+00,2022-09-12,451262,2022-12-12
3,ac543274a1527329,NaN,2.065440e+08,9.999570e+07,2023-04-30,ac543274a1527329,2023-04-30
4,95511,004,2.800000e+09,1.826667e+09,2022-08-11,95511,2022-08-11


In [34]:
loans_df['summa'].dtype

dtype('float64')

In [35]:
today = pd.to_datetime(datetime.today())

# difference between today and contract_date calculating in days
loans_df['day_since_last_loan'] = (today - loans_df['contract_date']).dt.days

In [36]:
# In case no claims at all, then put -3 as a value of this feature.
loans_df.loc[loans_df['claim_id'].isnull() | (loans_df['claim_id'] == ''), 'day_since_last_loan'] = -3

# In case no loans at all, then put -1 as a value of this feature.
loans_df.loc[(loans_df['loan_summa'] == 0) | (loans_df['loan_summa'].isnull()), 'day_since_last_loan'] = -1

In [37]:
# final result
loans_df

,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date,day_since_last_loan
0,522530,003,5.000000e+08,0.000000e+00,2020-02-13,609965,2020-02-17,-1
1,35163,053,5.100000e+08,0.000000e+00,2020-12-15,35163,2020-12-21,-1
2,500873,055,5.000000e+08,0.000000e+00,2022-09-12,451262,2022-12-12,-1
3,ac543274a1527329,NaN,2.065440e+08,9.999570e+07,2023-04-30,ac543274a1527329,2023-04-30,560
4,95511,004,2.800000e+09,1.826667e+09,2022-08-11,95511,2022-08-11,822
...,...,...,...,...,...,...,...,...
543,12144992,005,1.930000e+09,1.177375e+09,2022-10-21,3382076,2022-10-21,751
544,553516633,063,4.600000e+08,4.227725e+08,2023-09-10,553516633,2023-09-10,427
545,79365,014,3.400000e+08,3.305556e+08,2023-12-26,79365,2023-12-26,320
546,18410,004,4.916470e+07,0.000000e+00,2019-03-04,18410,2019-03-04,-1
